# lecture 6 - PoS, Committee-based blockchains

## Imports and helper functions

In [1]:
#Import statements
import hashlib as hasher
import random
import time
import matplotlib.pyplot as plt

def hashbits(input):
    hash_obj = hasher.sha256()
    inputbytes = input.encode()
    #print(type(inputbytes))
    hash_obj.update(inputbytes)
    hashbytes = hash_obj.digest()
    return ''.join(f'{x:08b}' for x in hashbytes)

def hash(input):
    hash_obj = hasher.sha256()
    inputbytes = input.encode()
    #print(type(inputbytes))
    hash_obj.update(inputbytes)
    return hash_obj.hexdigest()

In [2]:
####### Drawing blockchain, not important
from IPython.display import HTML, display
from IPython.html.widgets.interaction import interact

def maxHeight(parent):
  if len(parent.children) == 0:
    return parent.height 
  max = 0
  for child in parent.children:
    m = maxHeight(child)
    if m> max:
      max = m
  return max
  

def drawBlockchain(parent, level, html, parentLevel, childN = 0, total = 0):
  color = "#AEF751"
  if parentLevel!=-1:
    color = "#7EDBF6"
  parent.children.sort(key=lambda x: (maxHeight(x)), reverse=True)
  xx = childN
  level += childN
  html += '<g>'
  html += '<rect x="'+str(30+ 100*parent.height)+'" y="'+str(30+ 100*level)+'" width="60" height="60" stroke="black" stroke-width="1" fill="'+color+'" />'
  html += '<text x="'+str((60+ 100*parent.height))+'" y="'+str((60+ 100*level))+'" dominant-baseline="middle" text-anchor="middle" font-family="Verdana" font-size="10" font-weight="bold" fill="black">'+str(parent.creator.name)+'</text>'
  if parentLevel != -1:
    if (parent.previous.children.index(parent)) == 0:
      html += '<line stroke-width="1px" stroke="#000000"  x1='+str(30+ 100*parent.height)+' y1="'+str(60+ 100*level)+'" x2="'+str(95+ 100*parent.previous.height)+'" y2="'+str(60+ 100*parentLevel)+'" style="marker-end: url(#markerArrow)"/>'
    else:
      html += '<line stroke-width="1px" stroke="#000000"  x1='+str(30+ 100*parent.height)+' y1="'+str(60+ 100*level)+'" x2="'+str(65+ 100*parent.previous.height)+'" y2="'+str(95+ 100*parentLevel)+'" style="marker-end: url(#markerArrow)"/>'
  html += '</g>'
  l = level
  childN = 0
  for child in parent.children:
    html,n, t = drawBlockchain(child, l, html, level, childN, total)
    if n > 0:
      childN += n
    if t > 0:
      total += t
    l = l+1
  return html, childN+ len(parent.children)-1, total+ len(parent.children)-1


def show(bc):
  htmll = ""
  html = ""
  htmll, n, t = drawBlockchain(bc.chain[0], 0, html, -1)
  html = '<svg height="'+str(115*(n+1))+'" width="'+str(130*maxHeight(bc.chain[0]))+'">'
  html += '<defs><marker id="markerArrow" markerWidth="10" markerHeight="10" refX="2" refY="6" orient="auto"><path d="M2,2 L2,11 L10,6 L2,2" style="fill: #000000;" /> </marker> </defs>'
  html += htmll
  html += '</svg>'
  display(HTML(html))

/usr/local/lib/python3.7/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


## Proof of Stake

### Exercise 1: Proof of Stake (Peer Coin)

PoS is one of the widely used alternatives for PoW. In the PoS the mining power is replaced with stakes.  

- Below there are stubs for Blockchain, Block, and Minter. Each block has a property as the timestamp it was created on. Note the hash_block function.
- The difficulty now is not the number of zeros, it is a decimal number. 
- Complete `isSmaller` function to return 'True' if the first 15 bits of the block hash is valid based on blockchain difficulty and minter stake. 
- Update the `isSmaller` function. Assume the state of a Minter has increased by 1 for every block he has created so far. You can use the `checkMiner` function.
- (optional) Try to adjust the `isSmaller` function, based on the total stake that is in the blockchain.

In [3]:
max = int("1111111111111111",2)

In [5]:
class Block:
    def __init__(self, data, creator=None, previous=None, time=0):
        self.data = data
        if previous is None:
            self.previous = None
            self.previous_hash = ""
            self.creator = Minter(0 , "0")
            self.height = 0
        else:
            self.previous = previous
            self.previous_hash = previous.hash
            self.creator = creator
            self.height = previous.height+1
        self.timestamp = time
        self.hash = self.hash_block()
        self.children = []

    def pos_hash(self):
        return hashbits(self.creator.name + self.previous_hash + str(self.timestamp))

    def hash_block(self):
        return hashbits(self.creator.name + str(self.data) + self.previous_hash + str(self.timestamp))

    def print(self):
      print(self.data + " "+ self.creator.name + " " + str(self.height))
        
class Blockchain:
    def __init__(self, genesis_data, difficulty):
        self.chain = []
        self.chain.append(Block(genesis_data))
        self.difficulty = difficulty
        self.size = 0
        self.totalStake = 0

    def lastBlock(self):
      max = self.chain[0].height
      for block in self.chain:
        if block.height > max:
          max = block.height
      maxes = [block for block in self.chain if block.height == max]
      r = random.choices(maxes, k=1)
      return r[0]

    def lastBlocks(self):
      max = self.chain[0].height
      for block in self.chain:
        if block.height > max:
          max = block.height
      maxes = [block for block in self.chain if block.height == max]
      return maxes
        
    def add(self, newBlock):
        self.chain.append(newBlock)
        newBlock.previous.children.append(newBlock)
        self.size +=1
        #newBlock.creator.stake+=1
    
    def isSmaller(self, hashStr, creator):
      #add this function
      # use int(hashStr[0:15],2) to convert the first 15 bits to int 
      # compare it with the difficulty, multiplicated by the creators stake
      if int(hashStr[0:15],2) < self.difficulty * (creator.stake+self.checkMiner(creator)):
        return True
      return False

    
    def checkMiner(self, miner, last=None):
      if last == None:
        last = self.lastBlock()
      count = 0
      while last!=None:
        if last.creator == miner:
          count += 1
        last = last.previous
      return count

class Minter:
  def __init__(self, stake, name, blockchain=None):
    self.stake = stake
    self.name = name
    self.blockchain = blockchain
    
    if self.blockchain != None:
      self.blockchain.totalStake += self.stake
      self.lastBlock = blockchain.lastBlock()

  def updateLast(self):
    latest = self.blockchain.lastBlock()
    if latest.height > self.lastBlock.height:
        self.lastBlock = latest

  def PoSSolver(self, seconds):
    newBlock = Block(str(self.blockchain.size), self, self.lastBlock, seconds)
    h = newBlock.pos_hash()
    if self.blockchain.isSmaller(h,self):
      self.blockchain.add(newBlock)
      self.lastBlock = newBlock

bc = Blockchain("0" , 0.1)
m1 = Minter(10 ,"m1", bc)
m2 = Minter(15, "m2", bc)
m3 = Minter(20, "m3", bc)
m4 = Minter(12, "m4", bc)
start_time = time.time()
while bc.size < 10:
  seconds = (time.time() - start_time)
  m1.updateLast()
  m1.PoSSolver(seconds)
  m2.updateLast()
  m2.PoSSolver(seconds)
  m3.updateLast()
  m3.PoSSolver(seconds)
  m4.updateLast()
  m4.PoSSolver(seconds)

print("m1 initial stake: {} , m1 blocks found {}".format(m1.stake,bc.checkMiner(m1)))
print("m2 initial stake: {} , m2 blocks found {}".format(m2.stake,bc.checkMiner(m2)))
print("m3 initial stake: {} , m3 blocks found {}".format(m3.stake,bc.checkMiner(m3)))
print("m4 initial stake: {} , m4 blocks found {}".format(m4.stake,bc.checkMiner(m4)))

m1 initial stake: 10 , m1 blocks found 1
m2 initial stake: 15 , m2 blocks found 2
m3 initial stake: 20 , m3 blocks found 5
m4 initial stake: 12 , m4 blocks found 2


In [ ]:
show(bc)

### Exercise 2: Long range attacks

Below is an attacker trying to do a long range attack.

- Try the attack multible times, with different difficulty. Try also changing the base string `"0"`. How many blocks does the attackers chain get?

In [6]:
class Blockchain:
    def __init__(self, genesis_data, difficulty):
        self.chain = []
        self.chain.append(Block(genesis_data))
        self.difficulty = difficulty
        self.size = 0
        self.totalStake = 0

    def lastBlock(self):
      max = self.chain[0].height
      for block in self.chain:
        if block.height > max:
          max = block.height
      maxes = [block for block in self.chain if block.height == max]
      r = random.choices(maxes, k=1)
      return r[0]

    def lastBlocks(self):
      max = self.chain[0].height
      for block in self.chain:
        if block.height > max:
          max = block.height
      maxes = [block for block in self.chain if block.height == max]
      return maxes
        
    def add(self, newBlock):
        self.chain.append(newBlock)
        newBlock.previous.children.append(newBlock)
        self.size +=1
        #newBlock.creator.stake+=1
    
    def isSmaller(self, hashStr, creator):
      if int(hashStr[0:15],2) < self.difficulty * (creator.stake + self.checkMiner(creator, creator.lastBlock)):
        return True
      return False

    
    def checkMiner(self, miner, last=None):
      if last == None:
        last = self.lastBlock()
      count = 0
      while last!=None:
        if last.creator == miner:
          count += 1
        last = last.previous
      return count


class MinterLR:
  def __init__(self, stake, name, blockchain=None):
    self.stake = stake
    self.name = name
    self.blockchain = blockchain
    if self.blockchain != None:
      # start from genesis block
      self.lastBlock = blockchain.chain[0]

  def PoSSolver(self, seconds):
    #add this function
    newBlock = Block(str(self.blockchain.size), self, self.lastBlock, seconds)
    h = newBlock.pos_hash()
    if self.blockchain.isSmaller(h,self):
      self.blockchain.add(newBlock)
      self.lastBlock = newBlock

#create blockchain as above
# try changing the "0" here for a different example.
bc = Blockchain("0" , 0.1)
m1 = Minter(10 ,"m1", bc)
m2 = Minter(15, "m2", bc)
m3 = Minter(20, "m3", bc)
m4 = Minter(12, "m4", bc)
seconds = 0

#try increasing the 20 here to run a longer example
while bc.size < 20:
  seconds += 1
  m1.updateLast()
  m1.PoSSolver(seconds)
  m2.updateLast()
  m2.PoSSolver(seconds)
  m3.updateLast()
  m3.PoSSolver(seconds)
  m4.updateLast()
  m4.PoSSolver(seconds)


lastsecond = bc.lastBlock().timestamp

lrm = MinterLR(20, "lr", bc)
for second in range(0,lastsecond):
  lrm.PoSSolver(second)

print("Main chain size: {}".format(bc.lastBlock().height))
print("Attack chain size: {}".format(lrm.lastBlock.height))

Main chain size: 20
Attack chain size: 4


In [ ]:
show(bc)

### Exercise 3: PoW minter

Below is a PoW minter. If this minter finds a block, he manipulates the data, to find also the next block. 

- Play with the example, by trying to find the next block in more iterations, or creating a longer chain.
- Try to remove the attack, by updating the `hash_block` function.

In [7]:
class PoWMinter(Minter):
  def __init__(self, stake, name, blockchain=None):
    super().__init__(stake, name, blockchain)
    
  def PoSSolver(self, seconds):
    newBlock = Block(str(self.blockchain.size), self, self.lastBlock, seconds)
    h = newBlock.pos_hash()
    if self.blockchain.isSmaller(h,self):
      for i in range(1000):
        newBlockx = Block(str(self.blockchain.size) + str(i) , self, self.lastBlock, seconds)
        nextBlock = Block(str(self.blockchain.size), self, newBlockx, seconds+1)
        h = nextBlock.pos_hash()
        if self.blockchain.isSmaller(h,self):
          print("will find also next block")
          self.blockchain.add(newBlockx)
          self.lastBlock = newBlockx
          return
      self.blockchain.add(newBlock)
      self.lastBlock = newBlock

bc = Blockchain("0" , 0.2)
m1 = PoWMinter(10 ,"m1", bc)
m2 = Minter(10, "m2", bc)
m3 = Minter(10, "m3", bc)
m4 = Minter(10, "m4", bc)
seconds = 0

while bc.size < 60:
  seconds += 1
  m1.updateLast()
  m2.updateLast()
  m3.updateLast()
  m4.updateLast()
  
  m1.PoSSolver(seconds)
  m2.PoSSolver(seconds)
  m3.PoSSolver(seconds)
  m4.PoSSolver(seconds)


print("m1 initial stake: {} , m1 blocks found {}".format(m1.stake,bc.checkMiner(m1)))
print("m2 initial stake: {} , m2 blocks found {}".format(m2.stake,bc.checkMiner(m2)))
print("m3 initial stake: {} , m3 blocks found {}".format(m3.stake,bc.checkMiner(m3)))
print("m4 initial stake: {} , m4 blocks found {}".format(m4.stake,bc.checkMiner(m4)))

will find also next block
will find also next block
m1 initial stake: 10 , m1 blocks found 18
m2 initial stake: 10 , m2 blocks found 13
m3 initial stake: 10 , m3 blocks found 7
m4 initial stake: 10 , m4 blocks found 22


In [ ]:
show(bc)

### Exercise 4: Nothing at stake

Below is an example of the nothing at stake attack for you to play around with.



In [8]:
class MinterNS:
  def __init__(self, stake, name, blockchain=None):
    self.stake = stake
    self.name = name
    self.blockchain = blockchain
    if self.blockchain != None:
      self.lastBlocks = blockchain.lastBlocks()
      self.lastBlock = blockchain.lastBlock()
      self.activeForks = []
      self.activeForks.append(self.lastBlock)

  def updateLast(self):
    latests = self.blockchain.lastBlocks()
    if latests[0].height > self.lastBlocks[0].height:
        self.lastBlocks = latests
        for b in self.lastBlocks:
          self.activeForks.append(b)

  def PoSSolver(self, seconds):
    #add this function
    count = 0
    for lastBlock in self.activeForks[:]:
      newBlock = Block(str(self.blockchain.size), self, lastBlock, seconds)
      self.lastBlock = lastBlock
      h = newBlock.hash_block()
      if self.blockchain.isSmaller(h,self):
        self.blockchain.add(newBlock)
        self.activeForks.remove(lastBlock)
        count += 1
    #if len(self.lastBlocks) > 1:
      #print(count)

bc = Blockchain("0" , 10)
m1 = MinterLR(10 ,"m1lr", bc)
m2 = MinterNS(10, "m2", bc)
m3 = Minter(10, "m3", bc)
m4 = Minter(10, "m4", bc)
start_time = time.time()
while bc.size < 10:
  seconds = []
  for i in range (10):
    seconds.append(time.time() - start_time)
  for second in seconds:
    m2.PoSSolver(second)
    m3.PoSSolver(second)
    m4.PoSSolver(second)
  m2.updateLast()
  m3.updateLast()
  m4.updateLast()
while bc.size < 60:
  seconds = []
  for i in range (20):
    seconds.append(time.time() - start_time)
  for second in seconds:
    m1.PoSSolver(second)
  for second in seconds:
    m2.PoSSolver(second)
  for second in seconds:
    m3.PoSSolver(second)
  for second in seconds:
    m4.PoSSolver(second)
  m2.updateLast()
  m3.updateLast()
  m4.updateLast()

print(m1.stake)
print(m2.stake)
print(m3.stake)
print(m4.stake)

10
10
10
10


In [ ]:
show(bc)

In [ ]:
int("1111111111111111",2)

65535